In [6]:
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import smolvlm_picture_description,PictureDescriptionVlmOptions

import time
from pathlib import Path

from docling_core.types.doc import PictureItem
pipeline_options = PdfPipelineOptions()
pipeline_options.generate_picture_images = True
pipeline_options.images_scale = 2
pipeline_options.do_picture_classification = True
pipeline_options.do_picture_description = True
# pipeline_options.picture_description_options = (
#     repo_id = "ChatDOC/OCRFlux-3B"
#     # smolvlm_picture_description  # <-- the model choice
#)
pipeline_options.picture_description_options = PictureDescriptionVlmOptions(
    repo_id = "ChatDOC/OCRFlux-3B",  # <-- add here the Hugging Face repo_id of your favorite VLM
    prompt="Extract what in document.",
)


converter = DocumentConverter(format_options={
    InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
})

In [7]:
path = "/Users/jbc/Documents/SevenSix/etl_data_extraction/data/s3_downloads/営業本部/メーカー別/finisar/05. 代理店証明書/国立大学法人九州大学_代理店証明書.pdf"
DOC_SOURCE= Path(path)
doc = converter.convert(DOC_SOURCE).document


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  9.17it/s]


In [11]:
doc.export_to_markdown()

'logo\n\n<!-- image -->'

In [9]:
from docling_core.types.doc.document import PictureDescriptionData
from IPython import display

html_buffer = []
# display the first 5 pictures and their captions and annotations:
for pic in doc.pictures[:5]:
    html_item = (
        f"Picture {pic.self_ref}"
        f' '
        f"Caption{pic.caption_text(doc=doc)}"
    )
    for annotation in pic.annotations:
        if not isinstance(annotation, PictureDescriptionData):
            continue
        html_item += (
            f"Annotations ({annotation.provenance}){annotation.text}\n"
        )
    html_buffer.append(html_item)
display.HTML("".join(html_buffer))